### 4.1 머신러닝 네 가지 분류
- 지도학습
- 비지도학습
- 자기 지도 학습: 지도 학습인데 레이블을 경험적인(heuristic) 알고리즘을 사용해 입력 데이터로부터 생성
- 지도/자기지도/비지도 경계 모호할 때가 있음
- 강화학습

#### 분류와 회귀에서 사용하는 용어
- 타깃: 정답. 외부 데이터 소스에 근거하여 모델이 완벽하게 예측해야 하는 값
- 클래스: 분류 문제에서 선택 가능한 레이블의 집합
- 레이블: 분류 문제에서 클래스 할당의 구체적인 사례.
- 참값/꼬리표: 데이터셋에 대한 모든 타깃
- 벡터 회귀: 타깃이 연속적인 값의 '집합'인 작업. 여러 개의 값에 대한 회귀를 한다면 벡터 회귀.
- 미니 배치(또는 배치): 일반적으로 8~128개. 메모리 할당 용이하게끔 2의 거듭제곱으로 많이 함.

### 4.2 머신러닝 모델 평가
- 머신러닝의 목표는 처음 본 데이터에서 잘 작동하는 일반화된 모델을 얻는 것.

#### 4.2.1 훈련, 검증, 테스트 세트
- 정보 누설(information leak): 검증 세트의 모델 성능에 기반해 모델의 하이퍼파라미터를 조정할 때마다 검증 데이터에 관한 정보가 모델로 새어나감. \
따라서 테스트 세트 필요.
- 단순 홀드아웃 검증
- K-겹 교차 검증
- 셔플링을 사용한 반복 K-겹 교차 검증

#### 4.2.2 기억해야 할 것
- 대표성 있는 데이터
- 시간의 방향
- 데이터 중복

#### K-겹 교차검증 예제
https://3months.tistory.com/321

In [13]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import numpy as np

# Function to create model, required for KerasClassifier
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(4, input_dim=8, activation='relu'))
    model.add(Dense(4, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# load pima indians dataset
dataset = numpy.loadtxt("D:/숙탯 2기/pima-indians-diabetes.csv", delimiter=",")

# split into input (X) and output (Y) variables
X = dataset[:,0:8] 
Y = dataset[:,8]

# create model
model = KerasClassifier(build_fn=create_model, epochs=150, batch_size=10, verbose=0)   # 케라스 모델과 사이킷런이 호환되도록
kfold = KFold(n_splits=4, shuffle=True, random_state=seed) 
results = cross_val_score(model, X, Y, cv=kfold)

print(results)
print(np.mean(results))

C:\Users\judyj\AppData\Local\Temp\ipykernel_9256\3310952859.py:32: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, epochs=150, batch_size=10, verbose=0)   # 케라스 모델과 사이킷런이 호환되도록


[0.63541669 0.66666669 0.703125   0.76041669]
0.6914062649011612


#### 반복 K-겹 교차 검증 예제
https://tensorflow.blog/2017/12/27/%EB%B0%98%EB%B3%B5-%EA%B5%90%EC%B0%A8-%EA%B2%80%EC%A6%9D/

In [14]:
# 기본(KFold)

from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold
from sklearn.linear_model import LogisticRegression

iris = load_iris()
logreg = LogisticRegression(solver='liblinear', multi_class='auto', max_iter=1000)

In [15]:
kfold = KFold(n_splits=5)
scores = cross_val_score(logreg, iris.data, iris.target, cv=kfold)
scores, scores.mean()

(array([1.        , 0.93333333, 0.43333333, 0.96666667, 0.43333333]),
 0.7533333333333333)

In [16]:
# RepeatedKFold

from sklearn.model_selection import RepeatedKFold
rkfold = RepeatedKFold(n_splits=5, n_repeats=5, random_state=42)
scores = cross_val_score(logreg, iris.data, iris.target, cv=rkfold)
scores, scores.mean()

(array([1.        , 0.93333333, 0.93333333, 0.96666667, 0.96666667,
        0.96666667, 0.93333333, 1.        , 1.        , 0.83333333,
        0.93333333, 0.9       , 0.96666667, 0.9       , 0.93333333,
        0.96666667, 1.        , 0.93333333, 0.93333333, 0.93333333,
        0.96666667, 0.9       , 1.        , 0.93333333, 0.93333333]),
 0.9466666666666668)

In [17]:
# 계층별 교차검증(StratifiedKFold)

skfold = StratifiedKFold(n_splits=5)
scores = cross_val_score(logreg, iris.data, iris.target, cv=skfold)
scores, scores.mean()

(array([1.        , 0.96666667, 0.93333333, 0.9       , 1.        ]),
 0.9600000000000002)

In [18]:
# RepeatedStratifiedKFold

from sklearn.model_selection import RepeatedStratifiedKFold
rskfold = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=42)
scores = cross_val_score(logreg, iris.data, iris.target, cv=rskfold)
scores, scores.mean()

(array([0.96666667, 1.        , 0.9       , 0.93333333, 1.        ,
        0.96666667, 0.96666667, 0.96666667, 0.96666667, 0.93333333,
        0.93333333, 1.        , 1.        , 0.93333333, 0.96666667,
        0.96666667, 1.        , 0.96666667, 0.9       , 0.96666667,
        0.96666667, 0.9       , 0.96666667, 1.        , 0.96666667]),
 0.9613333333333334)

### 4.3 데이터 전처리, 특성 공학, 특성 학습

#### 4.3.1 신경망을 위한 데이터 전처리
- 벡터화
- 정규화: 네트워크를 쉽게 학습시키려면 데이터가 작은 값(일반적으로 0~1 사이)을 취하고, 균일(모든 특성이 대체로 비슷한 범위 가짐)해야 함.
- 누락된 값 다루기: 0, 평균, 중간값 대체 셋 다 교차검증으로 확인해보는 게 좋음.

#### 4.3.2 특성 공학


### 4.4 과대적합과 과소적합

#### 4.4.1 네트워크 크기 축소
- 너무 많은 용량과 충분하지 않은 용량 사이의 절충점 찾아야 함.
- 작은 네트워크가 기본 네트워크보다 더 나중에 과대적합 시작되고, 과대적합 시작되었을 때 성능이 더 천천히 감소.
- 용량이 많은 네트워크일수록 더 빠르게 훈련데이터를 모델링할 수 있지만 과대적합에 더 민감.

#### 4.4.2 가중치 규제 추가
- 간단한 모델이 복잡한 모델보다 과대적합 덜함.
- 네트워크 복잡도에 제한을 두어 가중치가 작은 값을 가지도록 규제.

```python
from keras import regularizers

# L1 규제
regularizers.l1(0.001)

# L2 규제
regularizers.l2(0.001)

# L1과 L2 규제 병행
regularizers.l1_l2(l1=0.001, l2=0.001)
```

#### 4.4.3 드롭아웃 추가
- 훈련하는 동안 무작위로 층의 일부 출력 특성을 제외(0으로 만듦)
- 테스트 단계에서는 어떤 유닛도 드롭아웃 x. 대신 층의 출력을 드롭아웃 비율에 비례해 줄여줌.
- 케라스에서는 층의 출력 바로 뒤에 드롭아웃 추가

### 4.5 보편적인 머신 러닝 작업 흐름